In [1]:
import numpy as np
import pandas as pd
import os
import multiprocessing as mul
from multiprocessing import Process

## IMPORTING AND SORTING THE DATA

WE HAVE CHOSEN ICECUBE 2008-18 DATA FOR THIS STUDY


THE ICE CUBE DATA SET HAS 1134450 NEUTRINO EVENTS


We select neutrino events with Energy >= 100TeV = 10^5 GeV

i.e log10(E/GeV) > 5


There are 192107 such neutrino events in this data

The ms pulsars are taken from the ATNF Catalogue

There are 441 pulsars (as of May 2022 when the study started)

All the pulsars lie in the declination range of -87 to +87 degrees

In [2]:
####
#### IMPORTING AND SPLITTING ICDATA $$$


path = "/media/darkwake/VIB2/Project-IceCube/icecube_10year_ps/events"
filenames = ["IC40_exp.csv", "IC59_exp.csv","IC79_exp.csv", "IC86_I_exp.csv", "IC86_II_exp.csv",
"IC86_III_exp.csv", "IC86_IV_exp.csv", "IC86_V_exp.csv", "IC86_VI_exp.csv", "IC86_VII_exp.csv"]
file = filenames[0]
f = open(os.path.join(path, file), 'r')
lines = f.readlines()
column=lines[0].split()
column.pop(0)
content = []
for file in filenames:
    f = open(os.path.join(path, file), 'r')
    lines = f.readlines()
    for line in lines[1:]:
        content.append(line.split())
    f.close()
icdata = pd.DataFrame(content, columns=column)
icdata['log10(E/GeV)'] = [float(i) for i in icdata['log10(E/GeV)']]

icdata = icdata.sort_values('log10(E/GeV)')
icdata = icdata.reset_index()
icdata = icdata.drop('index', axis=1)
icdata2 = icdata[icdata['log10(E/GeV)'] > 5]
icdata2 = icdata2.reset_index()
icdata2 = icdata2.drop('index', axis=1)
icdata2

#IMPORTING MSPDATA
f = open("/media/darkwake/VIB2/Project-IceCube/10milsecpsr.txt", 'r')
lines = f.readlines()

content=[]
column=lines[3].split()
for line in lines[:]:
    content.append(line.split())
    #the INITAL DATABASE IS CLUTTERED SO WE REMOVE THE NULL COLUMNS AND OTHER CLUTTER
mspdata = pd.DataFrame(content).drop(range(0,6)).dropna().drop([2,6,8,10,11,13,14], axis=1)
f.close()
line = []
lines = []

mspdata.columns = column
column = []
content=[]
mspdata.dropna(inplace=True)
mspdata = mspdata.reset_index()
mspdata = mspdata.drop(["index", "#"], axis=1)

In [14]:
msra = [float(i) for i in mspdata['RAJD']]
msdec = [float(i) for i in mspdata['DECJD']]
extensions = 441 - len(icdata2)%441
icra = [float(i) for i in icdata2['RA[deg]']]
icdec = [float(i) for i in icdata2['Dec[deg]']]
icra.extend([0]*extensions)
icdec.extend([0]*extensions)

Selecting events which are at an angular distance < $20^\circ$ from a particular ms psr source = $N$

In [15]:
def hvovec(lon1, lat1, lon2, lat2):

    #Convert decimal degrees to Radians:
    lon1 = np.deg2rad(lon1)
    lat1 = np.deg2rad(lat1)
    lon2 = np.deg2rad(lon2)
    lat2 = np.deg2rad(lat2)

    #Implementing Haversine Formula: 
    dlon = np.subtract(lon2, lon1)
    #dlat = np.subtract(lat2, lat1)

    a = np.add(np.multiply(np.sin(lat1), np.sin(lat2)), np.multiply(np.multiply(np.cos(lat1), np.cos(lat2)), np.cos(dlon)))

    return np.abs(np.rad2deg(np.arccos(a)))

st2a = []
lg = int(len(icra)/len(msra))
p = len(msra)    
for k in range(lg):
    ilo = icra[k * p : p * k + p]
    ila = icdec[k * p  : p * k + p]
    lo =[]
    la = []
    for j in range(p):#441
        lo = [ilo[(i + j)%p] for i in range(0,p)]
        la = [ila[(i + j)%p] for i in range(0,p)]
        st2a.append(hvovec(msra, msdec, lo, la))
        if k == lg - 1:
            st2a[-1][extensions:] = None

In [126]:
#all_angles = load.t2a2b(icra,icdec,msra,msdec,extensions,which='a')
lg = int(len(icra)/len(msra))
p = len(msra)  
#for k in range(lg):
k = list(range(0,lg))
def mul_angle_finder(k):
    st2a = []
    ilo = icra[k * p  :p * k + p]
    ila = icdec[k * p  :p * k + p]
    lo =[]
    la = []
    for j in range(p):#441
        lo = [ilo[(i + j)%p] for i in range(0,p)]
        la = [ila[(i + j)%p] for i in range(0,p)]
        st2a.append(hvovec(lo, la, msra, msdec))
        if k == lg - 1:
            st2a[-1][extensions:] = None
    return st2a
pool = mul.Pool()
op_async = pool.map_async(mul_angle_finder, range(0,lg))
all_angles_o = op_async.get()

In [127]:
all_angles = []
for k in range(len(all_angles_o)):
    for j in range(p):
        p_all_ang = [all_angles_o[k][j][i] for i in range(p)]
    all_angles.append(p_all_ang) 

In [135]:
max(all_angles_o[0][2])

170.6911147999076

In [ ]:
def S_i(i, s, x, xs, sg):
    return np.exp((x - xs[s])**2 / (2 * (sg[i])**2)) / (2 * np.pi * (sg[i] ** 2))

$B_i$ = No.of Events in a dataset/Solid angle average acrpss a band of +-6 deg in declination around the $i^{th}$ source

Solid angle span = $2\pi\cos(\sin(\delta + 3) - sin(\delta - 3))$

We estimate the background PDF by calculating the ratio of average number of events per steradian at each
declination and the total number of events. 

In [ ]:
def b_ivec(lon1, lat1, lon2, lat2):

    #Convert decimal degrees to Radians:
    lon1 = np.radians(lon1)
    lat1 = np.radians(lat1)
    lon2 = np.radians(lon2)
    lat2 = np.radians(lat2)

    #Implementing Haversine Formula: 
    dlon = np.subtract(lon2, lon1)
    #dlat = np.subtract(lat2, lat1)


    a = np.add(np.multiply(np.sin(lat1), np.sin(lat2)), np.multiply(np.multiply(np.cos(lat1), np.cos(lat2)), np.cos(dlon)))

    return (np.rad2deg(np.arccos(a)), a)

In [ ]:
evts_p_srad = []
l_ic = len(icdata2)
for i in range(len(mspdata)):
#returns $B_i$ as defined above
#def B_i(i):

    r = []
    psra = float(mspdata['RAJD'][i])
    psdec = float(mspdata['DECJD'][i])
    pssa = np.cos(np.cos(np.deg2rad(psdec) + np.cos(np.deg2rad(3))) - np.cos(np.deg2rad(psdec) - np.cos(np.deg2rad(3))))
    for j in range(l_ic):
        if icdec[j] > np.deg2rad(psdec) - pssa and icdec[j] < np.deg2rad(psdec) + pssa:
            r.append(j)
    evts_p_srad.append([i,r])

    

n_s = Signal Events from point source

In [ ]:
def Pr(s, S, B, N, ns):
    return ns * S / N + (1 - ns/N)*B
#for i in N
# find P[i]

In [ ]:
def L_i(s, S, B, N, ns):
    L = 1
    for i in range(len(N)):
        L *= Pr(s, S[i], B[i],  N[i], ns)
    return L

In [136]:
def TS(s, S, B, N):
    ts = []
    for i in range(0,4):
        ts.append(2 * np.log10(L_i(s, S, B, N, i)/L(s, S, B, N, 0)))
    return ts

In [ ]:
#Find ns such that TS is maximized